### Using BERT

https://medium.com/@samia.khalid/bert-explained-a-complete-guide-with-theory-and-tutorial-3ac9ebc8fa7c

https://www.analyticsvidhya.com/blog/2021/12/multiclass-classification-using-transformers/

In [1]:
SEED = 1

import pandas as pd
import sklearn
import seaborn as sns

PATH_SENTENCE_TEXT = r"../dataset/concat_sentence_text.csv"
PATH_SEGMENT_TEXT = r"../dataset/concat_segment_text.csv"

In [2]:
df = pd.read_csv(PATH_SENTENCE_TEXT)
df.head()

,sentence_text,practice,modality
0,"IP ADDRESS, COOKIES, AND WEB BEACONS",Identifier_Cookie_or_similar_Tech_1stParty,PERFORMED
1,"IP ADDRESS, COOKIES, AND WEB BEACONS",Identifier_IP_Address_1stParty,PERFORMED
2,"IP addresses will be collected, along with inf...",Identifier_IP_Address_1stParty,PERFORMED
3,The information that our products collect incl...,Identifier_Cookie_or_similar_Tech_1stParty,PERFORMED
4,The information that our products collect incl...,Identifier_IP_Address_1stParty,PERFORMED


In [3]:
df["sentence_text"] = df["sentence_text"].astype("string")
df["practice"] = df["practice"].astype("category")
df["practice"] = df["practice"].astype("category")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18829 entries, 0 to 18828
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   sentence_text  18829 non-null  string  
 1   practice       18829 non-null  category
 2   modality       18829 non-null  object  
dtypes: category(1), object(1), string(1)
memory usage: 315.2+ KB


In [4]:
# Match each privacy practice to a unique int
df["practice_int"] = df["practice"].cat.rename_categories(list(range(0, len(df["practice"].unique()))))

In [5]:
cat_array = pd.unique(df[["practice", "practice_int"]].values.ravel("K"))
# 58 different categories
cat_array[ :58]

array(['Identifier_Cookie_or_similar_Tech_1stParty',
       'Identifier_IP_Address_1stParty',
       'Identifier_Cookie_or_similar_Tech_3rdParty',
       'Demographic_3rdParty', 'Demographic_Age_3rdParty',
       'Contact_E_Mail_Address_1stParty', 'Demographic_Age_1stParty',
       'Contact_Phone_Number_1stParty', 'Contact_Postal_Address_1stParty',
       'Facebook_SSO', 'SSO', 'Contact_Address_Book_1stParty',
       'Contact_Password_1stParty', 'Demographic_1stParty',
       'Identifier_Ad_ID_1stParty', 'Identifier_Device_ID_1stParty',
       'Location_1stParty', 'Identifier_IMEI_1stParty',
       'Identifier_IMSI_1stParty', 'Identifier_SSID_BSSID_1stParty',
       'Location_Cell_Tower_1stParty', 'Location_IP_Address_1stParty',
       'Location_3rdParty', 'Location_Cell_Tower_3rdParty',
       'Location_IP_Address_3rdParty', 'Identifier_IP_Address_3rdParty',
       'Contact_E_Mail_Address_3rdParty', 'Identifier_MAC_1stParty',
       'Demographic_Gender_1stParty', 'Contact_1stParty',
 

In [6]:
df_cat_mapping = pd.DataFrame(
    {"id" : cat_array[58: ],
    "practice" : cat_array[ :58]
    })

df_cat_mapping.head()

,id,practice
0,27,Identifier_Cookie_or_similar_Tech_1stParty
1,35,Identifier_IP_Address_1stParty
2,28,Identifier_Cookie_or_similar_Tech_3rdParty
3,17,Demographic_3rdParty
4,19,Demographic_Age_3rdParty


In [7]:
df_bert = pd.DataFrame(
    {"id" : range(0, len(df)),
    "label" : df["practice_int"],
    "alpha" : ["a"] * len(df),
    "text" : df["sentence_text"]
    })

df_bert.head()

,id,label,alpha,text
0,0,27,a,"IP ADDRESS, COOKIES, AND WEB BEACONS"
1,1,35,a,"IP ADDRESS, COOKIES, AND WEB BEACONS"
2,2,35,a,"IP addresses will be collected, along with inf..."
3,3,27,a,The information that our products collect incl...
4,4,35,a,The information that our products collect incl...


In [8]:
from sklearn.model_selection import train_test_split